### Project Kojak

** The problem  **

We will attempt to identify amboguously defined words - words that are homographs (spelled the same, but with multiple meanings) and determine the exact meaning of the word from a context window.

Here we attempt to do this in a few stages
1. train a word embedding on some training corpus using skip-gram (Here we use 1000 sholarly research papers) 
2. identify common homographs and extract the various context windows
3. interpret the context windows as vectors in the embedding space and appy a clustering algorith (DBSCAN). Each cluster is interpreted as a distinct definition of the homograph. Each cluster then is representative vector.
4. apply to a test corpus - match context of given homograph to most similar group.


### This notebook

This notebook takes a set of training data, trains a word embedding model, and saves it for further use.

In [1]:
import gensim
import json
import os
import re
import time
from nltk.corpus import stopwords
from pprint import pprint



Using Theano backend.


In [24]:
# Declare stopwords, preprocess the data from source file
# Define class MyPapers, which takes a json file, loads the appropriate text data,
# and preprocesses the text to create a list of tokenized "sentences"

stop = stopwords.words('english')
stop += ['?','!',':',';','[',']','[]','“' ]
stop += ['.', ',', '(', ')', "'", '"',"''",'""',"``",'”', '“', '?', '!', '’', 'et', 'al', 'al.']
stop = set(stop)

class MyPapers(object):
    # a memory-friendly way to load a large corpora
     def __init__(self, dirname):
            self.dirname = dirname
 
     def __iter__(self):
        with open(self.dirname) as data_file:    
            data = json.load(data_file)
        # iterate through all file names in our directory
        for paper in data:
            try:
                line = [word for word in paper['full_text'].lower().split() if word not in stop]
                line = [re.sub(r'[?\.,!:;\(\)“\[\]]',' ',l) for l in line]
                yield line
            except:
                print(paper)
                print("Empty document found")
                continue
                

## Extract Context Window 

In [3]:
# Declare what word we are searchig for
target = u'extract'

In [25]:
#Instantiate iterable on the data

#papers is an iterable of scholarly papers, tokenized for prcessing
papers = MyPapers('data/train_data.json') 



In [5]:
# Arguments: target word and a starting number
# Prints all context windows for the target word for 10 papers in 

def print_10_contexts(target, paper_start, target_corpus):
    
    paper_count = 0
    print(target.upper())
    while paper_count < 10:
    #for paper_num in range(paper_start,paper_start + 10 ):
        try:
            paper = target_corpus[paper_start + paper_count]
            paper_count += 1
            windows = generate_windows([paper],6)
            count = 1
        except:
            break
        print('\nPAPER {}'.format(paper_count))
        for w in windows:
            if w[0] == target:
                print('{}: {}'.format(count, make_sentence(w[1])))
                count += 1
    

In [6]:
# Takes a list of strings (words_list)
# Returns single string of all words in words_list seperated by a white space.

def make_sentence(words_list):
    return ''.join([word + ' ' for word in words_list]).encode('utf-8')
    

In [7]:
#paper_start = 50

#print_10_contexts(target, paper_start, target_corpus)

## Word embeddings

In [8]:
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from functools import reduce

In [9]:
#Initialize the model

model = gensim.models.word2vec.Word2Vec(sentences = papers, size=300, window=6, min_count=1, workers=4,sg=1)

In [10]:
model.corpus_count

2848

In [11]:
model.save("data/journal.txt")

In [12]:
vectors = model.wv

In [13]:
len(vectors.vocab)

201268

## identifying n-grams

In [26]:
phrases = gensim.models.phrases.Phrases(sentences = papers, min_count = 5, threshold = 150)

In [27]:
for phrase, score in phrases.export_phrases(papers):
    print(u'{0}   {1}'.format(phrase, score))

last decade   587.762357887
computer aided   544.708211144
design  cad    297.841432453
written consent   223.714887922
written consent   223.714887922
tumor resection   233.478980447
bone graft    237.982703395
squamous cell   1275.14073227
= 20    178.952751796
cell carcinoma   244.72397892
1 h   172.89111582
cardiac arrest   1695.01596958
ct scan   1135.12351628
bone grafts   1065.07013159
iliac crest   11257.3030303
tumor resection   233.478980447
direct indirect   399.820296049
take account   319.685899918
subject matter    185.036194587
emphasis placed   437.83161048
formative assessment   412.309655938
authentic  inquiry-based   12383.0333333
teaching learning    159.12964592
recent years    320.141311212
authentic  inquiry-based   12383.0333333
southeast asia    652.310798946
southeast asia   1551.35055293
daily life    216.807317621
authentic  inquiry-based   12383.0333333
authentic  inquiry-based   12383.0333333
puts it    205.017108168
subject matter    185.036194587
authent

Exception RuntimeError: 'generator ignored GeneratorExit' in <generator object __iter__ at 0x117faed70> ignored


UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 8: ordinal not in range(128)

In [30]:
"Andronov\u2013Hopf".encode('utf-8')

LookupError: unknown encoding: u

In [ ]:
bigram = gensim.models.phrases.Phraser(phrases)